In [1]:
import os
import delta
import mimesis
import pyspark
from pyspark.sql import SparkSession

In [2]:
def get_spark() -> SparkSession:
    builder = (
        pyspark.sql.SparkSession.builder
        .master("local[*]")
        .appName("TestApp")
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
        .config("spark.sql.catalog.spark_catalog","org.apache.spark.sql.delta.catalog.DeltaCatalog")
    )
    spark = delta.configure_spark_with_delta_pip(builder).getOrCreate()
    spark.sparkContext.setLogLevel("ERROR")
    return spark

In [3]:
def create_dataset(i: int = 100) -> list[dict]:
    fake = mimesis.Generic()
    output = [
        {
            "name": fake.person.name(),
            "surname": fake.person.surname(),
            "birthday": fake.datetime.date(1960, 2010),
            "email": fake.person.email(),
            "country": fake.address.country(),
            "state": fake.address.state(),
            "city": fake.address.city(),
        }
        for _ in range(i)
    ]
    return output

In [4]:
dir = os.getcwd()
spark = get_spark()

df = spark.createDataFrame(create_dataset(i=1_000_000))

df = df.select(
    df.name, df.surname, df.birthday, df.email, df.country, df.state, df.city
)
df.write.format("delta").mode("overwrite").save(dir + os.sep + "/data/people")
df.show()

:: loading settings :: url = jar:file:/Users/subhashpeshwa/miniconda/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/subhashpeshwa/.ivy2/cache
The jars for the packages stored in: /Users/subhashpeshwa/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-35e50d67-14ab-41e6-a0a1-9c2274febfed;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.4.0 in central
	found io.delta#delta-storage;2.4.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 189ms :: artifacts dl 6ms
	:: modules in use:
	io.delta#delta-core_2.12;2.4.0 from central in [default]
	io.delta#delta-storage;2.4.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |

+---------+----------+----------+--------------------+--------------------+------------+---------------+
|     name|   surname|  birthday|               email|             country|       state|           city|
+---------+----------+----------+--------------------+--------------------+------------+---------------+
|  Alfonzo|     Potts|2010-09-28|graduate2043@live...|          Kazakhstan| Mississippi|   Maple Valley|
|    Ernie|Livingston|1968-10-15|sexuality1818@exa...|              Sweden|South Dakota|        Salinas|
|Ellsworth|    Franco|1987-03-01|podcasts1806@yaho...|             Andorra| Mississippi|   Granite City|
|    Denis|     Foley|1973-12-05|accurately2094@pr...|              Norway|   Minnesota|         Mandan|
| Carolyne| Blackwell|1990-06-16|boulevard2074@duc...|            Guernsey|  Washington|         Arvada|
|    Kayce|     Black|1993-12-29|surveys1859@examp...|              Gambia|     Vermont|         Topeka|
|  William|      Byrd|1999-05-24|fighters1835@prot...| 

#### 